# prologue

### set up notebook and load package

In [ ]:
# load what we need
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp
import multiprocessing as mp

# data management

### datasets

Any dataset can be turned into a container by invoking the constructor found in the module structures.py, the only mandatory parameters are an object that can be input to pandas.DataFrame and a class column name. Your object needs either its own column names, or these should be passed as a list to the var_names parameter.

Several datasets are available as pre-prepared containers that hold the data and some meta-data that is used in the algorithm. If these pre-packaged sets have more than 10,000 rows then we've included some downsampled versions. You can see what there is by doing:

In [ ]:
# ignore the builtins, np, pd, urllib and cfg. The rest are dataset constructors that will load specific datasets.
if False:
    dir(ds)

You can see more information about each dataset with something like this for the adult dataset:

In [ ]:
if False:
    print(ds.adult().spiel)

An example of one dataset is given below. Note that the random_state propagates through other functions via the meta_data and is easily updated to allow alternative runs:

In [ ]:
if False:
    cardio = ds.cardio(random_state=123, project_dir=project_dir)
    meta_data = cardio.get_meta()
    meta_data['random_state']

The list of datasets used in "CHIRPS: Explaining Random Forest Classification" is:

In [ ]:
if False:
    datasets = [
            ds.adult,
            ds.bankmark,
            ds.car,
            ds.cardio,
            ds.credit,
            ds.german,
            ds.lending_tiny_samp,
            ds.nursery,
            ds.rcdv,
           ]

The list of datasets used in "Explaining Multi-class AdaBoost Classification in Medical Applications" is:

In [ ]:
if False:
    datasets = [
                ds.breast,
                ds.cardio,
                ds.diaretino,
                ds.heart,
                ds.mhtech14,
                ds.mh2tech16,
                ds.readmit,
                ds.thyroid,
                dsp.usoc2,
               ]

### Standardising train-test splitting across environments
To compare with methods in different environments, as we did in "CHIRPS: Explaining Random Forest Classification," you can maintain the same dataset splits. The train test data is split with the one-time random_state_splits and the splits are saved to csv in the project folders. This way you can do same model with different data splits, or same data splits with different model. This is what's happening behind the scenes:

In [ ]:
##### do not run this here
# train_index, test_index = mydata.get_tt_split_idx(random_state=random_state_splits)
# tt = mydata.tt_split(train_index, test_index)

All you need here is the following, to take the same train_index, test_index numbers to an external .csv file

# Experimental Runs
From here on, everything is automated behind the scenes to perform runs across several datasets and comparing different algorithms with CHIRPS. If you want to run CHIRPS and control the parameters and view results and performance directly, look at the CHIRPS Examples Notebook.

In [ ]:
# do the model tuning from scratch?
override_tuning = False

# Optional Memory and Computation Cost Management
# CHIRPS is time economical but memory intensive to compute for lots of instances at once.
forest_walk_async=True
chirps_explanation_async=True
n_cores = mp.cpu_count()-2

# How many instances from the test set do you want to explain?
# A number bigger than the test set will be interpreted as 'all'
n_instances = 1000
start_instance = 0 # here can opt to start at a specific instance, diagnostic if something crashes

# CHOOSE ONE
model = 'RandomForest'
# model = 'AdaBoost1' # SAMME
# model = 'AdaBoost2' # SAMME.R
# model = 'GBM'

# CHOOSE ONE OR MORE
do_CHIRPS = True
do_Anchors = False
do_dfrgTrs = False
do_lore = False

# list the dataset constructors you want to include
datasets = [
#         ds.adult,
#         ds.bankmark,
#         ds.car,
#         ds.cardio,
#         ds.credit,
#         ds.german,
        ds.lending_tiny_samp,
#         ds.nursery,
#        ds.rcdv,
       ]
# datasets = [ds.cervicalh] # here can opt for just one but it must be a list, e.g. datasets = [datasets[0]]

# location to save results
project_dir = '/datadisk/whiteboxing/benchmarks2'
# project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'

# set the random_state for various tasks. This is not the same as random.seed(), not system wide.
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

# To standardise train-test splitting across environments - writes external files
rp.export_data_splits(datasets=datasets, project_dir=project_dir, random_state_splits=random_state_splits)

# How much messaging to print to the screen?
verbose = True

# CHIRPS default parameters - see papers for details
merging_bootstraps = 20 # how many training bootstraps to test improvement in growing rule?
pruning_bootstraps = 20 # how many training bootstraps to test deterioration in pruning rule?
delta = 0.1 # pruning deterioration tolerance paramater

# this is here if you want to pass parameters to the methods
def benchmark_wrapper(do_CHIRPS=do_CHIRPS, do_Anchors=do_Anchors, do_dfrgTrs=do_dfrgTrs, do_lore=do_lore, **control):

    if do_CHIRPS:
        rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control.update({'method' : 'Anchors'})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    if do_dfrgTrs:
        control.update({'method' : 'defragTrees',
                    'Kmax' : 10, 'restart' : 100, 'maxitr' : 20})
        rp.do_benchmarking(benchmark_items, verbose, **control)

    if do_lore:
        control.update({'method' : 'lore'})
        rp.do_benchmarking(benchmark_items, verbose, **control)

# this is here to pass parameters to the model training and further parameters to CHIRPS
tuning = {'override' : override_tuning}
if model == 'RandomForest':
    tuning.update({'grid' : {'n_estimators': [(i + 1) * 200 for i in range(8)],
                            'max_depth' : [32]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           do_raw=True, do_discretise=do_Anchors,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)

    kwargs = {'support_paths' : 0.2, 'alpha_paths' : 0.5, 'disc_path_bins' : 8,
             'score_func' : 1, 'weighting' : 'kldiv',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps,
             'precis_threshold' : 0.99,
             'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
elif model == "AdaBoost1":
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=(do_CHIRPS or do_dfrgTrs), do_discretise=(do_Anchors or do_lore),
                                       start_instance=start_instance,
                                       verbose=verbose, n_cores=n_cores)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.1, 'alpha_paths' : 0.0,
             'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
             'score_func' : 1, 'weighting' : 'kldiv',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
elif model == 'AdaBoost2':
    algo = 'SAMME.R'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=True, do_discretise=do_Anchors,
                                       start_instance=start_instance,
                                       verbose=verbose, n_cores=n_cores)
    
    kwargs = {'paths_lengths_threshold' : 5,
                 'support_paths' : 0.01, 'alpha_paths' : 0.0,
                 'disc_path_bins' : 8, 'disc_path_eqcounts' : True,
                 'score_func' : 1, 'weighting' : 'kldiv',
                 'merging_bootstraps' : merging_bootstraps,
                 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    

    
else: # GBM - not fully implemented yet
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)